In [1]:
import pandas as pd
import numpy as np
import psycopg2
import os

# Read Raw Data

In [53]:


def fetchDataFromDB(year):
   
    dbname = "huq"
    username = "fa71f"
    password = os.environ.get('CRED')
    host = "172.20.67.57"
    port = "5432"

    conn= psycopg2.connect(
        dbname=dbname,
        host=host,
        user=username,
        password=password,
        port=port
    )
    print('Connected')

    cur=conn.cursor()

    
    # cur.execute(
    # f"""
    # SELECT device_id_hash as uid,COUNT(DISTINCT date_trunc('day', "timestamp")) AS m{month}
    # FROM by_year.huq_gla_{year}_v1_2
    # WHERE EXTRACT(MONTH FROM timestamp) = {month}
    # """
    # )

    cur.execute(
    f"""
    SELECT 
        device_iid_hash as uid,
        date_trunc('month', "timestamp") as month,
        date_trunc('day', "timestamp") as day,
        COUNT(DISTINCT date_trunc('day', "timestamp")) AS active_days
    FROM 
        by_year.huq_gla_{year}_v1_2
    GROUP BY 
        device_iid_hash, 
        date_trunc('month', "timestamp"),
        date_trunc('day', "timestamp")
    ORDER BY 
        uid, 
        month;
    """
    )

    # cur.execute(
    # f"""
    # SELECT 
    # device_iid_hash as uid, 
    # COUNT(DISTINCT DATE(timestamp)) AS active_days
    # FROM by_year.huq_gla_2023_v1_2 hgv 
    # GROUP BY device_iid_hash;
    # """
    # )

    result=cur.fetchall()

    cols=[desc[0] for desc in cur.description]
    
    cur.close()
    conn.close()


    
    return pd.DataFrame(result,columns=cols)

def getQuarter(x):
    months=x.values
    quarters=[]
    for month in months:
        if month>=1 and month<=3:
            quarters.append(1)
        elif month>=4 and month<=6:
            quarters.append(2)
        elif month>=7 and month<=9:
            quarters.append(3)
        elif month>=10 and month<=12:
            quarters.append(4)

    return quarters

In [54]:
year=2023
database='huq' #tamoco or huq
active_df=fetchDataFromDB(year)
active_df['month'] = active_df['month'].dt.month
active_df['dom'] = active_df['day'].dt.day
active_df

Connected


,uid,month,day,active_days,dom
0,/008SrnXmCoa6olg4SEv6A==,7,2023-07-02 00:00:00+00:00,1,2
1,00gwfbf/IByimE9jxi4P+g==,9,2023-09-04 00:00:00+00:00,1,4
2,00gwfbf/IByimE9jxi4P+g==,9,2023-09-06 00:00:00+00:00,1,6
3,01AeuNkOfQr3t07+V2AYtw==,8,2023-08-24 00:00:00+00:00,1,24
4,01AeuNkOfQr3t07+V2AYtw==,8,2023-08-25 00:00:00+00:00,1,25
...,...,...,...,...,...
159582,Z+ZpfzoVGDrIR5KU58yqXQ==,7,2023-07-25 00:00:00+00:00,1,25
159583,Z+ZpfzoVGDrIR5KU58yqXQ==,8,2023-08-10 00:00:00+00:00,1,10
159584,zZtTxhQAOkeu/KjDyJTqrQ==,4,2023-04-29 00:00:00+00:00,1,29
159585,zZUus3P3Xc5/RbStziG8iw==,12,2023-12-31 00:00:00+00:00,1,31


In [55]:
active_df.rename(columns={'active_days':'total_active_days'},inplace=True)
active_df

,uid,month,day,total_active_days,dom
0,/008SrnXmCoa6olg4SEv6A==,7,2023-07-02 00:00:00+00:00,1,2
1,00gwfbf/IByimE9jxi4P+g==,9,2023-09-04 00:00:00+00:00,1,4
2,00gwfbf/IByimE9jxi4P+g==,9,2023-09-06 00:00:00+00:00,1,6
3,01AeuNkOfQr3t07+V2AYtw==,8,2023-08-24 00:00:00+00:00,1,24
4,01AeuNkOfQr3t07+V2AYtw==,8,2023-08-25 00:00:00+00:00,1,25
...,...,...,...,...,...
159582,Z+ZpfzoVGDrIR5KU58yqXQ==,7,2023-07-25 00:00:00+00:00,1,25
159583,Z+ZpfzoVGDrIR5KU58yqXQ==,8,2023-08-10 00:00:00+00:00,1,10
159584,zZtTxhQAOkeu/KjDyJTqrQ==,4,2023-04-29 00:00:00+00:00,1,29
159585,zZUus3P3Xc5/RbStziG8iw==,12,2023-12-31 00:00:00+00:00,1,31


In [56]:
active_df['quarter']=active_df.groupby(['uid','month'])['month'].transform(getQuarter)
active_df

,uid,month,day,total_active_days,dom,quarter
0,/008SrnXmCoa6olg4SEv6A==,7,2023-07-02 00:00:00+00:00,1,2,3
1,00gwfbf/IByimE9jxi4P+g==,9,2023-09-04 00:00:00+00:00,1,4,3
2,00gwfbf/IByimE9jxi4P+g==,9,2023-09-06 00:00:00+00:00,1,6,3
3,01AeuNkOfQr3t07+V2AYtw==,8,2023-08-24 00:00:00+00:00,1,24,3
4,01AeuNkOfQr3t07+V2AYtw==,8,2023-08-25 00:00:00+00:00,1,25,3
...,...,...,...,...,...,...
159582,Z+ZpfzoVGDrIR5KU58yqXQ==,7,2023-07-25 00:00:00+00:00,1,25,3
159583,Z+ZpfzoVGDrIR5KU58yqXQ==,8,2023-08-10 00:00:00+00:00,1,10,3
159584,zZtTxhQAOkeu/KjDyJTqrQ==,4,2023-04-29 00:00:00+00:00,1,29,2
159585,zZUus3P3Xc5/RbStziG8iw==,12,2023-12-31 00:00:00+00:00,1,31,4


In [57]:
active_df=pd.DataFrame(active_df.groupby(['uid','quarter']).size(),columns=['active_days']).reset_index()
active_df

,uid,quarter,active_days
0,++BRgQoWruuYPa9E8QAHrQ==,3,2
1,++E3j/UFIC0ZxfdM+8UeLw==,1,36
2,++UqjdR7ZOr7zRSUCf4Ruw==,1,1
3,++dwKPlD6xO1VrDwP6w3bQ==,2,1
4,++jFFIdOwr9oMNEL5qDp6Q==,1,1
...,...,...,...
15715,zyvVtQm9I2cLdLUslZbfTw==,2,2
15716,zz+Ahabd4FEjZIY6cysNQg==,1,43
15717,zz+Ahabd4FEjZIY6cysNQg==,2,13
15718,zz+Ahabd4FEjZIY6cysNQg==,3,7


In [58]:
active_df=active_df.pivot(index='uid',columns='quarter',values=['active_days']).add_prefix('quarter_').reset_index().fillna(0)
active_df.columns = [f'quarter_{col[1]}_active_days' if col[1] else col[0] for col in active_df.columns]
active_df.rename(columns={
    'quarter_quarter_1_active_days':'active_days_q1',
    'quarter_quarter_2_active_days':'active_days_q2',
    'quarter_quarter_3_active_days':'active_days_q3',
    'quarter_quarter_4_active_days':'active_days_q4'
},inplace=True)

active_df['total_active_days']=active_df['active_days_q1']+active_df['active_days_q2']+active_df['active_days_q3']+active_df['active_days_q4']

active_df

,uid,active_days_q1,active_days_q2,active_days_q3,active_days_q4,total_active_days
0,++BRgQoWruuYPa9E8QAHrQ==,0.0,0.0,2.0,0.0,2.0
1,++E3j/UFIC0ZxfdM+8UeLw==,36.0,0.0,0.0,0.0,36.0
2,++UqjdR7ZOr7zRSUCf4Ruw==,1.0,0.0,0.0,0.0,1.0
3,++dwKPlD6xO1VrDwP6w3bQ==,0.0,1.0,0.0,0.0,1.0
4,++jFFIdOwr9oMNEL5qDp6Q==,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...
10932,zy8a3Sdk+8IpHgcZfjrTwQ==,0.0,47.0,5.0,0.0,52.0
10933,zyKcK0VaHR30Py9BPoCHFA==,0.0,0.0,1.0,0.0,1.0
10934,zyYPUCs+TTvhktZ6/mqCnA==,0.0,2.0,0.0,0.0,2.0
10935,zyvVtQm9I2cLdLUslZbfTw==,0.0,2.0,0.0,0.0,2.0


In [59]:
active_df[active_df['uid']=='IYn8NMMLLHuHUhkp6tTuOw==']

,uid,active_days_q1,active_days_q2,active_days_q3,active_days_q4,total_active_days
3529,IYn8NMMLLHuHUhkp6tTuOw==,0.0,0.0,0.0,24.0,24.0


In [60]:
active_df.to_csv(f'U:\\Projects\\{database.title()}\\Faraz\\final_OD_work\\{year}\\active_days_stat_{year}.csv',index=False)